In [ ]:
import torch

print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # esto por si hay disponible una GPU si no usa la CPU
print("CUDA:", torch.cuda.is_available())

2.9.1+cpu
CUDA: False


## Integridad de los datos

In [49]:
import os

base_path = r"C:\Proyectos\ServicioSocial\contaminante\contaminante"
data_types = ["train", "valid", "test"]

for split in data_types:
    images_path = os.path.join(base_path, split, "images")
    labels_path = os.path.join(base_path, split, "labels")

    num_images = len([
        f for f in os.listdir(images_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])

    num_labels = len(os.listdir(labels_path))

    print(f"{split.upper()}:")
    print(f"  Imágenes: {num_images}")
    print(f"  Labels:   {num_labels}")


TRAIN:
  Imágenes: 3704
  Labels:   3704
VALID:
  Imágenes: 1236
  Labels:   1236
TEST:
  Imágenes: 1232
  Labels:   1232


- dataset personalizado para el uso de multi etiqueta en una sola imagen

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image


class ContaminantDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(images_dir))
        self.class_to_idx = {
            "plastico": 0,
            "vidrio": 1,
            "metal": 2,
            "organico": 3
        }

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]

        # Imagen
        img_path = os.path.join(self.images_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        # Label multi-label
        label_path = os.path.join(
            self.labels_dir,
            img_name.replace(".jpg", ".txt")
        )

        label = torch.zeros(len(self.class_to_idx), dtype=torch.float32)

        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = int(parts[0])   # 👈 SOLO el ID de clase
                label[class_id] = 1

        if self.transform:
            image = self.transform(image)

        return image, label


- preprosesado de loas imagenes para ser usadas por los modelos

In [51]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

- construccion de las retas a los apartados del dataset

In [52]:
train_dataset = ContaminantDataset(
    base_path +"/train/images",
    base_path+"/train/labels",
    transform=train_transform
)

val_dataset = ContaminantDataset(
    base_path+"/valid/images",
    base_path+"/valid/labels",
    transform=val_transform
)

test_dataset = ContaminantDataset(
    base_path+"/test/images",
    base_path+"/test/labels",
    transform=test_transform
)


- verificasion de la catidad de los datos con respecto a catidad de archivos

In [53]:
print("este es el tamaño del apartado de entrenamiento: " ,len(train_dataset))
print("este es el tamaño del apartado de validacion: ",len(val_dataset))
print("este es el tamaño del apartado de prueba: ", len(test_dataset))


este es el tamaño del apartado de entrenamiento:  3704
este es el tamaño del apartado de validacion:  1236
este es el tamaño del apartado de prueba:  1232


In [54]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False
)


## Cargar ResNet18 preentrenada

In [55]:
import torch
import torch.nn as nn
import torchvision.models as models


NUM_CLASSES = 4   # ajusta a tus contaminantes reales

model = models.resnet18(pretrained=True)

# Cambiamos la última capa
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



- Loss y Optimizer

In [56]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

- Verificación rápida del DataLoader

In [57]:
images, labels = next(iter(train_loader))

print(images.shape)   # (batch, 3, 224, 224)
print(labels.shape)   # (batch, NUM_CLASSES)


torch.Size([32, 3, 224, 224])
torch.Size([32, 4])


- Training loop

In [58]:
model.train()

for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()

    outputs = model(images)
    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    print(f"Loss: {loss.item():.4f}")
    break   # ⛔ solo una iteración para prueba


Loss: 0.6192
